In [1]:
FTRAIN = 'data/training.csv' #getting training data

In [2]:
import csv
import random

def parse_csv(mark):
    csvfile = open(FTRAIN, 'r')
    reader = csv.reader(csvfile, delimiter='\n')
    my_list = list(reader)
    del my_list[:1]
    array = []
    for entry in my_list:
        entry = ''.join(''.join(entry).partition(' ')[1:])
        entry = entry.split()
        entry.append(random.getrandbits(1)) #marking images with faces
        array.append(entry)
        
    return array    
    
array = parse_csv("1")

In [3]:
x_train = []
y_train = []

for entry in array:
    y_train.append(entry[len(entry) - 1])
    x_train.append(entry[0:len(entry) - 1])

In [35]:
import numpy

x_train = numpy.array(x_train)
x_train = x_train.reshape(len(x_train), len(x_train[1]))
x_train = x_train.astype('float32')
x_train /= 225

In [42]:
from keras.utils import np_utils # утилита для декодирования истенных значений (перевод в Унитарный/Двоичный код)

y_train = np_utils.to_categorical(y_train, 2) 

In [36]:
from lasagne import layers
from nolearn.lasagne import NeuralNet
from nolearn.lasagne import BatchIterator
import theano

In [37]:
height, width, deph = 96,96,1 #параметры изображение 28x28 в сером спектре
batch_size = 128 # рассматриваем 128 обучающий парметров на каждой итерации
num_epochs = 1 # итерируемся 20 раз по обучающей выборке 
hidden_size = 512 # по 512 нейронов в каждом (из двух) слоев НС
num_classes = 2 # 10 классов 

In [38]:
from keras.layers import Input, Dense # 2 типа слоев НС
from keras.models import Model  #класс для определения и обучений НС

inp = Input(shape=(9215,)) #на входе вектор размера 784
hidden_1 = Dense(hidden_size, activation ='relu')(inp) #первый скрытый ReLu слой
hidden_2 = Dense(hidden_size, activation = 'relu')(hidden_1) #второй скрытый ReLu слой
out = Dense(num_classes, activation='softmax')(hidden_2) # выходной softmax слой
# softmax превращает вектор действительных чисел в вектор вероятностей

model = Model(input = inp, output = out)
#model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))

In [39]:
model.compile(loss='categorical_crossentropy', #функция потерь крос - энтропии
             optimizer = 'adam', #отпимизатор Адама
              metrics = ['accuracy'] #точность - доля входных данных, отнесенных к правильному классу
             )

In [43]:
model.fit(x_train, y_train, batch_size = batch_size, nb_epoch = num_epochs)
#model.evaluate(X, y,verbose=1) #оценка модели на тестовом наборе 

Epoch 1/1
7049/7049 [==============================] - 3s - loss: 7.8414 - acc: 0.5035     